## Experiment 1: (2022-07-28~)
- - -
#### 0. Preprocessing / (Done on seperate file)
Urbansound8k Dataset i){8732(<=4s).wav files/10 folders(must be separated)} => Noise data    
MTG-jamendo Dataset i){}=> Music data   
For easy labeling, we need to save the labels along with the data ( X.npy = [ data , label ] )

#### 1. Initialize input pipeline


#### 2. Model initialization


#### 3. Training & Evaluation

- - -

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_addons as tfa

import pathlib

import librosa

import soundfile as sf

from tqdm import tqdm

from IPython.display import clear_output

autotune = tf.data.AUTOTUNE

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_virtual_device_configuration(gpu, [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*11)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

## 1. Initializing Input Pipeline
* First define the raw tf.data.Datasets.   
* Make a function that preprocesses the training datasets.
* Use tf.data.Dataset.map to create the fully processed dataset.
* Then make a function that finds the labels for each of the training datasets.   
* Use tf.data.Dataset.map to create the label Dataset.   
* Then combine them with tf.data.Dataset.zip.   
* Then Shuffle & Configure for performance

In [6]:
base_path = pathlib.Path('D:/전재원졸업연구/music-discern/Datasets/')

#Urbansound is nested in Datasets/Urbansound8K/audio/fold**
#Urbansound is already separated into batches, which need to be preserved.
urban_metadata_path = pathlib.Path(base_path/'Urbansound8K/metadata')
urban_audio_folders_path = pathlib.Path(base_path/'Urbansound8K/processed')
urban_audio_path_list = [x for x in urban_audio_folders_path.iterdir()]
temp = urban_audio_path_list.pop(1)
urban_audio_path_list.append(temp)

#Jamendo is nested in Datasets/MTGJamendo/audio
#Jamendo is not separated in batches, so we need to separate it into 10 batches, matching urbansound.
jamendo_metadata_path = pathlib.Path(base_path/'MTGJamendo/metadata')
jamendo_audio_folders_path = pathlib.Path(base_path/'MTGJamendo/processed')
jamendo_audio_path_list = [x for x in jamendo_audio_folders_path.iterdir()]
temp = jamendo_audio_path_list.pop(1)
jamendo_audio_path_list.append(temp)


#for later -> add a path for any .wav file that I want to observe.
test_subjects_path = pathlib.Path("./")

Train = [1,2,3,4,5,6,7,8,9]
Test = [10] #Seperation, for later evaluation in Training step B

print(urban_audio_path_list)
print(jamendo_audio_path_list)

[WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold1'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold2'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold3'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold4'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold5'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold6'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold7'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold8'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold9'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold10')]
[WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJamendo/processed/fold1'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJamendo/processed/fold2'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJam

In [ ]:
dataset_for_training_A = "empty"
dataset_for_training_B_train = "empty"
dataset_for_training_B_test = "empty"

dataset_for_training_A = tf.data.Dataset.list_files(str(urban_audio_folders_path/'*/*.npy'))
dataset_for_training_A = dataset_for_training_A.concatenate( tf.data.Dataset.list_files(str(jamendo_audio_folders_path/'*/*.npy')) )

for k in Train:
    temp_path_urban = str(urban_audio_path_list[k])
    temp_path_jam = str(jamendo_audio_path_list[k])
    if dataset_for_training_B_train == "empty":
        dataset_for_training_B_train = tf.data.Dataset.list_files(temp_path_urban + '/*.npy')
        dataset_for_training_B_train = dataset_for_training_B_train.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )
    else:
        dataset_for_training_B_train = dataset_for_training_B_train.concatenate( tf.data.Dataset.list_files(temp_path_urban + '/*.npy') )
        dataset_for_training_B_train = dataset_for_training_B_train.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )

for k in Test:
    temp_path_urban = str(urban_audio_path_list[k])
    temp_path_jam = str(jamendo_audio_path_list[k])
    if dataset_for_training_B_test == "empty":
        dataset_for_training_B_test = tf.data.Dataset.list_files(temp_path_urban + '/*.npy')
        dataset_for_training_B_test = dataset_for_training_B_test.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )
    else:
        dataset_for_training_B_test = dataset_for_training_B_test.concatenate( tf.data.Dataset.list_files(temp_path_urban + '/*.npy') )
        dataset_for_training_B_test = dataset_for_training_B_test.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )

## 2. Model initialization

## 3. Training & Evaluation

![](/Experiment1abstract.jpg?raw=true)